In [1]:
import numpy as np
from supportFunctions import weightsFromFraction, getDephasingTimes, weightedCrbTwoEchoes
import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, CustomJS, Title
from bokeh.palettes import viridis

Weights are calculated given an in-phase echo fraction $f= \frac{w_1}{w_1 +w_2}$ while also maintaining unit variance if the echoes are summed, i.e. $w_1^2 + w^2 = 1$:

\begin{equation}
    w_1 = \frac{f}  {\sqrt{\left(1-f\right)^2 + f^2}} \\
    w_2 = \frac{1-f}{\sqrt{\left(1-f\right)^2 + f^2}}
\end{equation}

In [2]:
numFrac = 256
numPF = 256
W = weightsFromFraction(np.linspace(0,1,numFrac))

output_notebook()
p = figure(height=200, width=200, toolbar_location=None, title='Weights')
l = p.line(np.linspace(0,1,numFrac), W[0], legend="w₁", color='navy')
l = p.line(np.linspace(0,1,numFrac), W[1], legend="w₂", color='chocolate')
show(p)


Loading BokehJS ...

In [6]:
ta = 3.6e-3 # Available acquisition time
B0 = 3 # Tesla

partialFourierFactors = np.linspace(start=0.5, stop=1, num=numPF)
dephasingTimes = np.empty(shape=(numPF,numFrac,2), dtype=np.float32)
firstEchoFractions = np.linspace(start=0, stop=1, num=numFrac)
CRB = np.empty(shape=(numPF,numFrac,2), dtype=np.float32) # [W, F] at 0% fat fraction

for nPF,PF in enumerate(partialFourierFactors):
    for nf, f in enumerate(firstEchoFractions):
        dephasingTimes[nPF, nf, :] = getDephasingTimes(ta, PF, f)
        weights = weightsFromFraction(f);
        CRB[nPF, nf] = weightedCrbTwoEchoes(B0, dephasingTimes[nPF, nf, :], weights)
p = figure(height=350, width=700,tools="hover", toolbar_location=None, title='Search space')
p.image(image=[CRB[:,:,0]], x=0, y=.5, dw=1, dh=.5, palette=viridis(10))
p.image(image=[CRB[:,:,1]], x=1, y=.5, dw=1, dh=.5, palette=viridis(10))
p.x_range.range_padding = p.y_range.range_padding = 0
from bokeh.transform import linear_cmap
#colormapper = linear_cmap(field_name = factbook["Birth rate"], palette=palette, low=min(factbook["Birth rate"]), high=max(factbook["Birth rate"]))

show(p)